### IMPORT TIM'S REPORTS

COMPLAINTS RECEIVED

In [1]:
import pandas as pd
import numpy as np

path = "C:\\Users\\dmehri\\Documents\\DATA\\2N Analysis\\"

df = pd.read_csv(path + "2N_complaints_received.csv")

df["Complaint Number"] = df["Complaint Number"].astype(str).map(str.strip)


df["Address"] = df["House Number"] + " " + df["Street Name"] + ", " + df["Borough Name"]

df["Address"] = df["Address"].astype(str).map(str.strip).map(str.upper)



df.columns = df.columns.str.replace('  ', ' ')
df.columns = df.columns.str.replace(' ', '_')
df["BIN_Number"] = df["BIN_Number"].astype(str).map(str.strip)


df = df.rename(columns={'Source_of_Complaint_Description': 'Source'})
cols = sorted(df.columns)
#print cols

subset = ['Address', 'BIN_Number', 'Community_Board', 
          'Complaint_Date', 'Complaint_Number']

df["Complaint_Date"] = df["Complaint_Date"].astype(str).map(str.strip).map(str.upper).str.replace(' 0:00', '')


df = df[subset]

#df.dtypes

In [2]:
df.head(2)

,Address,BIN_Number,Community_Board,Complaint_Date,Complaint_Number
0,"2701 GLENWOOD ROAD, BROOKLYN",3327331,314,3/30/2020,3748469
1,"57 ST MARKS PLACE, MANHATTAN",1006394,103,4/1/2020,1540015


INSPECTIONS

In [3]:
pd.set_option('chained_assignment', None)

di = pd.read_csv(path + "2N_inspections.csv")

di["Complaint Number"] = di["Complaint Number"].astype(str).map(str.strip)

di = di.drop('BIN Number', 1)

di.columns = di.columns.str.replace('  ', ' ')
di.columns = di.columns.str.replace(' ', '_')

disp = ['A1', 'A4', 'A5', 'A9', 'B1', 'B2', 'A8', 'A6',
                'H3','A3', 'L3', 'L2', 'L1', 'Y4', 'Y3', 'Y2', 
                'Y1', 'W1', 'H6']

di["Number_Violations"] = 0
di["Number_SWO"] = 0


di["Disposition_Code"] = di["Disposition_Code"].astype(str).map(str.strip)


count = 0
countswo = 0
for i in range(0, len(di) ):
    if di["Disposition_Code"][i] in disp:
        di["Number_Violations"][i] = 1
        count+=1
    if "STOP WORK" in di["Complaint_Disposition_Description"][i]:
        di["Number_SWO"][i] = 1
        countswo+=1

di["Inspection_Date"] = di["Inspection_Date"].astype(str).map(str.strip).map(str.upper).str.replace(' 0:00', '')

di = di.drop('Work_Days_to_Inspection', 1)

di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].astype(str).map(str.upper)
di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"] + " "
di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].str.replace(' F ', '')
di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].str.replace(' P ', '')

di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].str.replace(' N ', '')

di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].str.replace('[^\w\s]','')
di["Complaint_Disposition_Description"] = di["Complaint_Disposition_Description"].map(str.strip)

print "number of disp", count
print "number swo", countswo

print di.dtypes

number of disp 200
number swo 94
Complaint_Number                     object
Inspection_Date                      object
Disposition_Code                     object
Complaint_Disposition_Description    object
Number_Violations                     int64
Number_SWO                            int64
dtype: object


In [4]:
di.head(2)

,Complaint_Number,Inspection_Date,Disposition_Code,Complaint_Disposition_Description,Number_Violations,Number_SWO
0,1540015,4/2/2020,I2,NO VIOLATION WARRANTED FOR COMPLAINT AT TIME O...,0,0
1,1540018,4/3/2020,I2,NO VIOLATION WARRANTED FOR COMPLAINT AT TIME O...,0,0


BACKLOG DATA

In [5]:
db = pd.read_csv(path + "2N_backlog.csv")

db["Complaint Number"] = db["Complaint Number"].astype(str).map(str.strip)
db.columns = db.columns.str.replace('  ', ' ')
db.columns = db.columns.str.replace(' ', '_')

db.dtypes

Complaint_Number             object
Work_Days_Since_Complaint     int64
dtype: object

MERGE

In [6]:
print "merge complaints received with inspections"
print len(df)
df2 = pd.merge(df, di, how='left', on=['Complaint_Number'])
print "merge with backlog"
print len(df2)
df2 = pd.merge(df2, db, how='left', on=['Complaint_Number'])
print len(df2)

#df2 = pd.merge(df2, dgis, how='left', on=['BIN_Number'])

#print len(df2)

cols = sorted(df2.columns)
print cols

#print 
#print colsOld

merge complaints received with inspections
1494
merge with backlog
1547
1547
['Address', 'BIN_Number', 'Community_Board', 'Complaint_Date', 'Complaint_Disposition_Description', 'Complaint_Number', 'Disposition_Code', 'Inspection_Date', 'Number_SWO', 'Number_Violations', 'Work_Days_Since_Complaint']


In [7]:
#df2.to_csv(path + "test.csv", index=False)

DATA CLEANING

In [8]:
print "Data cleaning..."
from datetime import datetime
print str(datetime.now())

pd.set_option('chained_assignment', None)


print "Clean workdays since complaint"
df2["Work_Days_Since_Complaint"] = df2["Work_Days_Since_Complaint"].astype(str).map(str.strip)
#df2["Work_Days_Since_Complaint"] = df2["Work_Days_Since_Complaint"].str.replace('nan', 'NA')


#dates = set(df["Inspection_Date(s)"].tolist() )

df2["Inspection_Date"] = df2["Inspection_Date"].astype(str).map(str.strip)
df2["Inspection_Date"] = df2["Inspection_Date"].str.replace(' 0:00', '')
df2["Inspection_Date"] = df2["Inspection_Date"].str.replace('nan', 'Not Inspected Yet')


df2["Inspected"] = "YES"
df2["Number_Inspected"] = 0
df2["Number_Open_Complaints"] = 0

for i in range(0, len(df2)):
    if df2["Inspection_Date"][i] == "Not Inspected Yet":
        df2["Inspected"][i] = "NO"
        df2["Number_Open_Complaints"][i] = 1
    else:
        df2["Number_Inspected"][i] = 1

        
        
        
#df["Work_Days_Since_Complaint"] = df["Work_Days_Since_Complaint"].astype(str)

#df["Complaint_Date"] = df["Complaint_Date"].astype(str).map(str.strip)
#df["Complaint_Date"] = df["Complaint_Date"].str.replace(' 0:00', '')



print "DONE"



Data cleaning...
2020-04-20 10:20:33.733000
Clean workdays since complaint
DONE


WORK DAYS TO FIRST INSPECTION

In [9]:
#import pandas as pd
#import numpy as np

#d = pd.DataFrame({"Signin Date": ["2018-01-01", "2018-02-01"]})
#d["Signin Date"] = pd.to_datetime(d["Signin Date"])
#d['Signin Date Shifted'] = pd.DatetimeIndex(d['Signin Date']) + pd.DateOffset(months=1)

#d["bussDays"] = np.busday_count( d["Signin Date"].values.astype('datetime64[D]'), d['Signin Date Shifted'].values.astype('datetime64[D]'))
#print(d)

In [10]:
import numpy as np

binList = list(set(df2["BIN_Number"].tolist() ))

qn = pd.DataFrame(columns=('BIN_Number', 'Work_Days_to_1st_Inspection', 
                           'First_Complaint_Date', 'First_Inspection_Date'))


for i in range(0, len(binList)):
#for i in range(1, 2):
    
    df3 = df2[df2["BIN_Number"] == binList[i]]
    df3 = df3[["BIN_Number", "Complaint_Date", "Inspection_Date"]]
    df3 = df3[df3["Inspection_Date"] != "Not Inspected Yet"]
    
    if len(df3) > 0:
        df3['Complaint_Date'] = pd.to_datetime(df3['Complaint_Date'])
        df3['Inspection_Date'] = pd.to_datetime(df3['Inspection_Date'])
        df3 = df3.sort_values(by = 'Complaint_Date', ascending=True).reset_index(drop=True)
        df3["bussDays"] = np.busday_count( df3['Complaint_Date'][0], 
                                          df3['Inspection_Date'][0])
        
        df3["Complaint_Date"] = df3["Complaint_Date"].apply(lambda x: x.date())
        df3["Inspection_Date"] = df3["Inspection_Date"].apply(lambda x: x.date())
        
        df3["Complaint_Date"] = df3["Complaint_Date"].astype(str)
        df3["Inspection_Date"] = df3["Inspection_Date"].astype(str)

        complaint_dates = df3["Complaint_Date"].tolist()
        inspection_dates = df3["Inspection_Date"].tolist()
    

        qn.loc[i] = [binList[i],df3["bussDays"][0], complaint_dates[0], inspection_dates[0] ]

    
    
    #print df3
#df2 = pd.merge(df2, qn, how='left', on=['BIN_Number'])



CALCULATE BACKLOG IN WORKDAYS

In [11]:
#qnb = pd.DataFrame(columns=('BIN_Number', 'Work_Days_Since_Complaint'))

from datetime import datetime
from dateutil import parser


current_date =  datetime.now().date()

print current_date


df2["Work_Days_Since_Complaint"] = ""

for i in range(0, len(df2)):
#for i in range(0, 10):
    complaint_date= df2["Complaint_Date"][i]
    complaint_date = complaint_date.replace('/', '-')
    complaint_date = parser.parse(complaint_date)
    complaint_date = complaint_date.date()

    
    df2["Work_Days_Since_Complaint"][i] = np.busday_count( complaint_date, 
                                          current_date)
        
    


2020-04-20


In [12]:
df2.dtypes

Address                               object
BIN_Number                            object
Community_Board                        int64
Complaint_Date                        object
Complaint_Number                      object
Inspection_Date                       object
Disposition_Code                      object
Complaint_Disposition_Description     object
Number_Violations                    float64
Number_SWO                           float64
Work_Days_Since_Complaint             object
Inspected                             object
Number_Inspected                       int64
Number_Open_Complaints                 int64
dtype: object

AGGREGATE TO BIN LEVEL

In [13]:
print "Group and add..."
dfbase = df2[["BIN_Number", "Number_Violations", "Work_Days_Since_Complaint", "Number_SWO",
             "Number_Inspected", "Number_Open_Complaints"]]

dfbase["Work_Days_Since_Complaint"] = dfbase["Work_Days_Since_Complaint"].astype(int)
dfbase["Number_Complaints"] = 1
dfbase = (dfbase.groupby('BIN_Number').agg({'Number_Violations':'sum', 
                                            'Work_Days_Since_Complaint': 'mean',
                                            'Number_Complaints':'sum',
                                            'Number_Inspected':'sum',
                                            'Number_Open_Complaints':'sum',
                                            'Number_SWO':'sum'}).reset_index().rename(columns={'Number_Violations':'Number_Violations'}) )

print "Add address..."
ad = df2[["BIN_Number", "Address"]]
ad = ad.drop_duplicates(subset=['BIN_Number', 'Address']).reset_index(drop=True)
ad = ad.set_index('BIN_Number')['Address'].to_dict()
dfbase["Address"] = dfbase["BIN_Number"].map(ad)


print "Add community board..."
cb = df2[["BIN_Number", "Community_Board"]]
cb = cb.set_index('BIN_Number')['Community_Board'].to_dict()
dfbase["Community_Board"] = dfbase["BIN_Number"].map(cb)

print "Add complaint dates..."
cdates = df2[["BIN_Number", "Complaint_Date"]]
cdates =cdates.drop_duplicates(subset=['BIN_Number', 'Complaint_Date']).reset_index(drop=True)
f = cdates.groupby('BIN_Number')['Complaint_Date'].apply(lambda x: "{%s}" % ', '.join(x))
f = pd.DataFrame(f)
f = f.reset_index(drop=False)
f = f.set_index('BIN_Number')['Complaint_Date'].to_dict()
dfbase["Complaint_Date(s)"] = dfbase["BIN_Number"].map(f)

print "Add inspection dates..."
cdates = df2[["BIN_Number", "Inspection_Date"]]
cdates =cdates.drop_duplicates(subset=['BIN_Number', 'Inspection_Date']).reset_index(drop=True)
f = cdates.groupby('BIN_Number')['Inspection_Date'].apply(lambda x: "{%s}" % ', '.join(x))
f = pd.DataFrame(f)
f = f.reset_index(drop=False)
f = f.set_index('BIN_Number')['Inspection_Date'].to_dict()
dfbase["Inspection_Date(s)"] = dfbase["BIN_Number"].map(f)


print "Add complaint numbers..."
cdates = df2[["BIN_Number", "Complaint_Number"]]
cdates =cdates.drop_duplicates(subset=['BIN_Number', 'Complaint_Number']).reset_index(drop=True)
f = cdates.groupby('BIN_Number')['Complaint_Number'].apply(lambda x: "{%s}" % ', '.join(x))
f = pd.DataFrame(f)
f = f.reset_index(drop=False)
f = f.set_index('BIN_Number')['Complaint_Number'].to_dict()
dfbase["Complaint_Number(s)"] = dfbase["BIN_Number"].map(f)

print "Merge with first inspection and complaint dates"
print "len of dfbase before merge", len(dfbase)
dfbase = pd.merge(dfbase, qn, how='left', on=['BIN_Number'])
print "len of dfbase after merge", len(dfbase)

print "Add dispositions..."
dsp = df2[["BIN_Number", "Disposition_Code","Complaint_Disposition_Description"]]
dsp = dsp.reset_index(drop=True)
dsp["Disposition_Code"] = dsp["Disposition_Code"].astype(str)
dsp["Complaint_Disposition_Description"] = dsp["Complaint_Disposition_Description"].astype(str)

dsp["Complaint_Disposition_Description"] = dsp["Disposition_Code"] + " " + dsp["Complaint_Disposition_Description"]
dsp = dsp[["BIN_Number", "Complaint_Disposition_Description"]]
dsp =dsp.drop_duplicates(subset=['BIN_Number', 'Complaint_Disposition_Description']).reset_index(drop=True)
f = dsp.groupby('BIN_Number')['Complaint_Disposition_Description'].apply(lambda x: "{%s}" % ', '.join(x))
f = pd.DataFrame(f)
f = f.reset_index(drop=False)

f.Complaint_Disposition_Description = f.Complaint_Disposition_Description.str.replace('[^\w\s]','')

f = f.set_index('BIN_Number')['Complaint_Disposition_Description'].to_dict()
dfbase["Complaint_Disposition(s)"] = dfbase["BIN_Number"].map(f)

print "Add inspected yes or no..."
inspected = df2[df2["Inspected"] == "YES"]
dfbase["Building Inspected"] = "NO"

inspected = list(set(inspected["BIN_Number"].tolist() ))
countinspected = 0
for i in range(0, len(dfbase)):
    if dfbase["BIN_Number"][i] in inspected:
        dfbase["Building Inspected"][i] = "YES"
        countinspected+=1

print "Count inspected", countinspected

print "len of dfbase", len(dfbase)

print "Add lat/lon"
pathgis = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\"
dgis = pd.read_csv(pathgis + "Property BIN to Lat Lon.csv", low_memory=False)
dgis = dgis[["BIN", "LATITUDE", "LONGITUDE"]]
dgis = dgis.rename(columns={'BIN': 'BIN_Number', 
                            'LATITUDE': 'Latitude',
                           'LONGITUDE': 'Longitude'})
dgis["BIN_Number"] = dgis["BIN_Number"].astype(str).map(str.strip)
dfbase = pd.merge(dfbase, dgis, how='left', on=['BIN_Number'])
df2 =  pd.merge(df2, dgis, how='left', on=['BIN_Number'])
print len(dfbase)



print "DONE"

Group and add...
Add address...
Add community board...
Add complaint dates...
Add inspection dates...
Add complaint numbers...
Merge with first inspection and complaint dates
len of dfbase before merge 947
len of dfbase after merge 947
Add dispositions...
Add inspected yes or no...
Count inspected 943
len of dfbase 947
Add lat/lon
947
DONE


In [14]:
dfbase.head(1)

,BIN_Number,Number_SWO,Number_Open_Complaints,Number_Violations,Number_Complaints,Number_Inspected,Work_Days_Since_Complaint,Address,Community_Board,Complaint_Date(s),Inspection_Date(s),Complaint_Number(s),Work_Days_to_1st_Inspection,First_Complaint_Date,First_Inspection_Date,Complaint_Disposition(s),Building Inspected,Latitude,Longitude
0,1000793,0.0,0,0.0,4,4,5.75,"90 WASHINGTON STREET, MANHATTAN",101,"{4/7/2020, 4/15/2020, 4/16/2020}","{4/7/2020, 4/8/2020, 4/15/2020, 4/16/2020}","{1540391, 1540411, 1541093, 1541120}",0.0,2020-04-07,2020-04-07,I2 NO VIOLATION WARRANTED FOR COMPLAINT AT TIM...,YES,40.708069,-74.014847


CREATE DISPOSIIOTN TYPE CATEGORY

In [15]:
dfbase["Disposition Type"] = "none"
for i in range(0, len(dfbase)):
    for j in range( 0, len(disp) ):
        if disp[j] in dfbase["Complaint_Disposition(s)"][i]:
            dfbase["Disposition Type"][i] = "Violation"
    if "STOP WORK" in dfbase["Complaint_Disposition(s)"][i]:
        dfbase["Disposition Type"][i] = "Stop Work Order"
    elif dfbase["Disposition Type"][i] == "none" :
        dfbase["Disposition Type"][i] = "No Violation"


CREATE A BACKLOG MAP

In [16]:
dfback = df2.copy()
dfback = dfback[dfback["Inspected"] == "NO"].reset_index(drop=True)

dfback = dfback[["BIN_Number", "Complaint_Date", "Complaint_Number", 
                "Work_Days_Since_Complaint", "Latitude", "Longitude"]]

dfback["Toltal_Number_Open"] = 1

f = dfback.groupby('BIN_Number')['Complaint_Date'].apply(lambda x: "{%s}" % ', '.join(x))
X = pd.DataFrame(f)
X = X.reset_index(drop=False)

f = dfback.groupby('BIN_Number')['Complaint_Number'].apply(lambda x: "{%s}" % ', '.join(x))
X2 = pd.DataFrame(f)
X2 = X2.reset_index(drop=False)

X = pd.merge(X, X2, how='left', on=['BIN_Number'])


dfback = dfback[["BIN_Number", "Work_Days_Since_Complaint", "Toltal_Number_Open",
                 "Latitude", "Longitude"]]

dfback["Work_Days_Since_Complaint"] = dfback["Work_Days_Since_Complaint"].astype(int)

dfback = (dfback.groupby('BIN_Number').agg({'Work_Days_Since_Complaint':'mean',
                                     'Toltal_Number_Open':'sum',
                                     'Latitude': 'mean', 
                                     'Longitude':'mean'}).reset_index().rename(columns={'Work_Days_Since_Complaint':'Avg_Work_Days_Since_Complaint'}) )


dfback = pd.merge(dfback, X, how='left', on=['BIN_Number'])


ad = df2[["BIN_Number", "Address"]]
ad = ad.drop_duplicates(["BIN_Number"]).reset_index(drop=True)
print "len of dfback before merge", len(dfback)
dfback = pd.merge(dfback, ad, how='left', on=['BIN_Number'])
print "len dfack after merge", len(dfback)


dfback["Complaint_Date"] = dfback["Complaint_Date"].astype(str).str.replace('{', '')
dfback["Complaint_Date"] = dfback["Complaint_Date"].str.replace('}', '')

dfback["Complaint_Number"] = dfback["Complaint_Number"].astype(str).str.replace('{', '')
dfback["Complaint_Number"] = dfback["Complaint_Number"].str.replace('}', '')

dfback = dfback.rename(columns={'Complaint_Date': 'Complaint_Date(s)', 
                        'Complaint_Number': 'Complaint_Number(s)'})


dfback["Building Inspected"] = "NO"


len of dfback before merge 27
len dfack after merge 27


In [17]:
dfback.head(1)

,BIN_Number,Latitude,Toltal_Number_Open,Longitude,Avg_Work_Days_Since_Complaint,Complaint_Date(s),Complaint_Number(s),Address,Building Inspected
0,1014076,40.743359,1,-73.996643,12.0,4/2/2020,1540027,"200 7 AVENUE, MANHATTAN",NO


In [18]:
from datetime import datetime

print str(datetime.now())

dfbase["Complaint_Date(s)"] = dfbase["Complaint_Date(s)"].astype(str).str.replace('{', '')
dfbase["Complaint_Date(s)"] = dfbase["Complaint_Date(s)"].str.replace('}', '')

dfbase["Inspection_Date(s)"] = dfbase["Inspection_Date(s)"].astype(str).str.replace('{', '')
dfbase["Inspection_Date(s)"] = dfbase["Inspection_Date(s)"].str.replace('}', '')

dfbase["Complaint_Number(s)"] = dfbase["Complaint_Number(s)"].astype(str).str.replace('{', '')
dfbase["Complaint_Number(s)"] = dfbase["Complaint_Number(s)"].str.replace('}', '')

dfbase["Complaint_Disposition(s)"] = dfbase["Complaint_Disposition(s)"].astype(str).str.replace('{', '')
dfbase["Complaint_Disposition(s)"] = dfbase["Complaint_Disposition(s)"].str.replace('}', '')
dfbase["Complaint_Disposition(s)"] = dfbase["Complaint_Disposition(s)"].str.replace('nan nan', '')

dfbase.to_csv(path + "2N BIN Level.csv", index=False)
df2.to_csv(path + "2N Complaint Level.csv", index=False)
dfback.to_csv(path + "2N Backlog Map.csv", index=False)

print "DONE WITH ANALYSIS"

2020-04-20 10:20:46.802000
DONE WITH ANALYSIS


In [19]:
dfbase.dtypes

BIN_Number                      object
Number_SWO                     float64
Number_Open_Complaints           int64
Number_Violations              float64
Number_Complaints                int64
Number_Inspected                 int64
Work_Days_Since_Complaint      float64
Address                         object
Community_Board                  int64
Complaint_Date(s)               object
Inspection_Date(s)              object
Complaint_Number(s)             object
Work_Days_to_1st_Inspection    float64
First_Complaint_Date            object
First_Inspection_Date           object
Complaint_Disposition(s)        object
Building Inspected              object
Latitude                       float64
Longitude                      float64
Disposition Type                object
dtype: object